In [2]:
import numpy as np
import pandas as pd
import nltk
from underthesea import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [3]:
df = pd.read_csv(filepath_or_buffer='./ViLegalDoc.csv')
df

,content,label
0,Phạm vi điều chỉnh: Luật này quy định về việc ...,LUẬT DOANH NGHIỆP
1,"Đối tượng áp dụng: Doanh nghiệp. Cơ quan, tổ c...",LUẬT DOANH NGHIỆP
2,Áp dụng Luật Doanh nghiệp và luật khác. Trường...,LUẬT DOANH NGHIỆP
3,Bảo đảm của Nhà nước đối với doanh nghiệp và c...,LUẬT DOANH NGHIỆP
4,"Tổ chức chính trị, tổ chức chính trị - xã hội ...",LUẬT DOANH NGHIỆP
...,...,...
1227,Điều 204. Trường hợp đình công bất hợp pháp\n\...,Luật lao động
1228,Điều 205. Thông báo quyết định đóng cửa tạm th...,Luật lao động
1229,Điều 206. Trường hợp cấm đóng cửa tạm thời nơi...,Luật lao động
1230,Điều 207. Tiền lương và các quyền lợi hợp pháp...,Luật lao động


# DATA PROCESSING

In [4]:
# val: 22 samples per class
# test: 27 samples per class
# train: the rest

def train_val_test_split(df, class_name):
    temp_df = df[df['label'] == class_name]

    test_df = temp_df.sample(n=27, random_state=42)
    temp_df = temp_df[~temp_df.index.isin(test_df.index)]

    val_df = temp_df.sample(n=22, random_state=42)
    train_df = temp_df[~temp_df.index.isin(val_df.index)]
    return train_df, val_df, test_df

In [5]:
df['label'].unique()

array(['LUẬT DOANH NGHIỆP', 'LUẬT VIÊN CHỨC', 'LUẬT PHÒNG, CHỐNG MA TÚY',
       'LUẬT BẢO Vệ MÔI TRƯỜNG', 'LUẬT ĐẤU THẦU', 'LUẬT GIÁO DỤC',
       'LUẬT ĐẤT ĐAI', 'Luật xử lý vi phạm hành chính', 'Luật lao động'],
      dtype=object)

In [6]:
df_train_1, df_val_1, df_test_1 = train_val_test_split(df=df, class_name='LUẬT DOANH NGHIỆP')
df_train_2, df_val_2, df_test_2 = train_val_test_split(df=df, class_name='LUẬT VIÊN CHỨC')
df_train_3, df_val_3, df_test_3 = train_val_test_split(df=df, class_name='LUẬT PHÒNG, CHỐNG MA TÚY')
df_train_4, df_val_4, df_test_4 = train_val_test_split(df=df, class_name='LUẬT BẢO Vệ MÔI TRƯỜNG')
df_train_5, df_val_5, df_test_5 = train_val_test_split(df=df, class_name='LUẬT ĐẤU THẦU')
df_train_6, df_val_6, df_test_6 = train_val_test_split(df=df, class_name='LUẬT GIÁO DỤC')
df_train_7, df_val_7, df_test_7 = train_val_test_split(df=df, class_name='LUẬT ĐẤT ĐAI')
df_train_8, df_val_8, df_test_8 = train_val_test_split(df=df, class_name='Luật xử lý vi phạm hành chính')
df_train_9, df_val_9, df_test_9 = train_val_test_split(df=df, class_name='Luật lao động')

In [7]:
df_train = pd.concat(objs=[df_train_1, df_train_2, df_train_3, df_train_4, df_train_5, df_train_6, df_train_7, df_train_8, df_train_9])
df_val = pd.concat(objs=[df_val_1, df_val_2, df_val_3, df_val_4, df_val_5, df_val_6, df_val_7, df_val_8, df_val_9])
df_test = pd.concat(objs=[df_test_1, df_test_2, df_test_3, df_test_4, df_test_5, df_test_6, df_test_7, df_test_8, df_test_9])

len(df_train), len(df_val), len(df_test)

(791, 198, 243)

In [8]:
stopword_file = "vietnamese-stopwords.txt"
with open(stopword_file, "r", encoding="utf-8") as f:
    stopword_content = f.read()

stop_words = stopword_content.splitlines()

In [9]:
# lower-casing - chuyen ve chu thuong
# tokenizing
# stopword removing
# numeric removing

def text_processing(content):
    cleaned_content = content.lower()
    cleaned_content = re.sub('[^a-zA-Zà-ỹĂÂĐÊÔƠƯơư]', ' ', cleaned_content)
    cleaned_content = word_tokenize(cleaned_content)
    cleaned_content = [word for word in cleaned_content if word not in stop_words]
    cleaned_content = ' '.join(cleaned_content)
    cleaned_content = re.sub(' +', ' ', cleaned_content)

    return  cleaned_content

In [10]:
df_train['cleaned_text'] = df_train['content'].apply(text_processing)
df_train

,content,label,cleaned_text
2,Áp dụng Luật Doanh nghiệp và luật khác. Trường...,LUẬT DOANH NGHIỆP,áp dụng luật doanh nghiệp luật trường hợp luật...
3,Bảo đảm của Nhà nước đối với doanh nghiệp và c...,LUẬT DOANH NGHIỆP,bảo đảm nhà nước đối với doanh nghiệp chủ sở h...
5,Quyền của doanh nghiệp. Tự do kinh doanh ngành...,LUẬT DOANH NGHIỆP,quyền doanh nghiệp tự do kinh doanh ngành nghề...
8,Người đại diện theo pháp luật của doanh nghiệp...,LUẬT DOANH NGHIỆP,đại diện pháp luật doanh nghiệp đại diện pháp ...
11,Điều 15. Trách nhiệm của người đại diện theo ủ...,LUẬT DOANH NGHIỆP,trách nhiệm đại diện ủy quyền chủ sở hữu thành...
...,...,...,...
1227,Điều 204. Trường hợp đình công bất hợp pháp\n\...,Luật lao động,trường hợp đình công bất hợp pháp trường hợp đ...
1228,Điều 205. Thông báo quyết định đóng cửa tạm th...,Luật lao động,thông báo quyết định đóng cửa tạm thời làm việ...
1229,Điều 206. Trường hợp cấm đóng cửa tạm thời nơi...,Luật lao động,trường hợp cấm đóng cửa tạm thời làm việc thời...
1230,Điều 207. Tiền lương và các quyền lợi hợp pháp...,Luật lao động,tiền lương quyền lợi hợp pháp lao động thời gi...


In [11]:
df_test['cleaned_text'] = df_test['content'].apply(text_processing)
df_test

,content,label,cleaned_text
62,"Điều 66. Tiền lương, thù lao, thưởng và lợi íc...",LUẬT DOANH NGHIỆP,tiền lương thù lao thưởng lợi ích chủ tịch hội...
40,"Điều 44. Chi nhánh, văn phòng đại diện và địa ...",LUẬT DOANH NGHIỆP,chi nhánh văn phòng đại diện địa điểm kinh doa...
93,Điều 97. Trách nhiệm của Chủ tịch và thành viê...,LUẬT DOANH NGHIỆP,trách nhiệm chủ tịch thành viên hội đồng thành...
18,Điều 22. Hồ sơ đăng ký công ty cổ phần. 1. Giấ...,LUẬT DOANH NGHIỆP,hồ sơ đăng ký công ty cổ phần giấy đề nghị đăn...
81,Điều 85. Cơ cấu tổ chức quản lý công ty trách ...,LUẬT DOANH NGHIỆP,cơ cấu tổ chức quản lý công ty trách nhiệm hữu...
...,...,...,...
1180,"Điều 157. Cấp, cấp lại, gia hạn, thu hồi giấy ...",Luật lao động,gia hạn thu hồi giấy phép lao động giấy xác nh...
1125,Điều 102. Khấu trừ tiền lương\n\n1. Người sử d...,Luật lao động,khấu trừ tiền lương sử dụng lao động khấu trừ ...
1097,Điều 66. Nguyên tắc thương lượng tập thể\n\nTh...,Luật lao động,nguyên tắc thương lượng tập thể thương lượng t...
1092,"Điều 61. Học nghề, tập nghề để làm việc cho ng...",Luật lao động,học nghề tập nghề làm việc sử dụng lao động họ...


In [12]:
df_val['cleaned_text'] = df_val['content'].apply(text_processing)
df_val

,content,label,cleaned_text
32,Điều 36. Định giá tài sản góp vốn. 1. Tài sản ...,LUẬT DOANH NGHIỆP,định giá tài sản góp vốn tài sản góp vốn đồng ...
1,"Đối tượng áp dụng: Doanh nghiệp. Cơ quan, tổ c...",LUẬT DOANH NGHIỆP,đối tượng áp dụng doanh nghiệp cơ quan tổ chức...
69,Điều 73. Công bố thông tin: Công ty trách nhiệ...,LUẬT DOANH NGHIỆP,công bố thông tin công ty trách nhiệm hữu hạn ...
6,Nghĩa vụ của doanh nghiệp. Doanh nghiệp xã hội...,LUẬT DOANH NGHIỆP,nghĩa vụ doanh nghiệp doanh nghiệp xã hội đáp ...
77,Điều 81. Chủ tịch công ty. 1. Chủ tịch công ty...,LUẬT DOANH NGHIỆP,chủ tịch công ty chủ tịch công ty chủ sở hữu c...
...,...,...,...
1094,Điều 63. Tổ chức đối thoại tại nơi làm việc\n\...,Luật lao động,tổ chức đối thoại làm việc đối thoại làm việc ...
1058,Điều 27. Kết thúc thời gian thử việc\n\n1. Khi...,Luật lao động,kết thúc thời gian thử kết thúc thời gian thử ...
1210,Điều 187. Thẩm quyền giải quyết tranh chấp lao...,Luật lao động,thẩm quyền giải quyết tranh chấp lao động cá n...
1140,Điều 117. Kỷ luật lao động\n\nKỷ luật lao động...,Luật lao động,kỷ luật lao động kỷ luật lao động quy định tuâ...


In [13]:
df_train.drop(columns=['content'], inplace=True)
df_train = df_train[['cleaned_text', 'label']]
df_train

,cleaned_text,label
2,áp dụng luật doanh nghiệp luật trường hợp luật...,LUẬT DOANH NGHIỆP
3,bảo đảm nhà nước đối với doanh nghiệp chủ sở h...,LUẬT DOANH NGHIỆP
5,quyền doanh nghiệp tự do kinh doanh ngành nghề...,LUẬT DOANH NGHIỆP
8,đại diện pháp luật doanh nghiệp đại diện pháp ...,LUẬT DOANH NGHIỆP
11,trách nhiệm đại diện ủy quyền chủ sở hữu thành...,LUẬT DOANH NGHIỆP
...,...,...
1227,trường hợp đình công bất hợp pháp trường hợp đ...,Luật lao động
1228,thông báo quyết định đóng cửa tạm thời làm việ...,Luật lao động
1229,trường hợp cấm đóng cửa tạm thời làm việc thời...,Luật lao động
1230,tiền lương quyền lợi hợp pháp lao động thời gi...,Luật lao động


In [14]:
df_test.drop(columns=['content'], inplace=True)
df_test = df_test[['cleaned_text', 'label']]
df_test

,cleaned_text,label
62,tiền lương thù lao thưởng lợi ích chủ tịch hội...,LUẬT DOANH NGHIỆP
40,chi nhánh văn phòng đại diện địa điểm kinh doa...,LUẬT DOANH NGHIỆP
93,trách nhiệm chủ tịch thành viên hội đồng thành...,LUẬT DOANH NGHIỆP
18,hồ sơ đăng ký công ty cổ phần giấy đề nghị đăn...,LUẬT DOANH NGHIỆP
81,cơ cấu tổ chức quản lý công ty trách nhiệm hữu...,LUẬT DOANH NGHIỆP
...,...,...
1180,gia hạn thu hồi giấy phép lao động giấy xác nh...,Luật lao động
1125,khấu trừ tiền lương sử dụng lao động khấu trừ ...,Luật lao động
1097,nguyên tắc thương lượng tập thể thương lượng t...,Luật lao động
1092,học nghề tập nghề làm việc sử dụng lao động họ...,Luật lao động


In [15]:
df_val.drop(columns=['content'], inplace=True)
df_val = df_val[['cleaned_text', 'label']]
df_val

,cleaned_text,label
32,định giá tài sản góp vốn tài sản góp vốn đồng ...,LUẬT DOANH NGHIỆP
1,đối tượng áp dụng doanh nghiệp cơ quan tổ chức...,LUẬT DOANH NGHIỆP
69,công bố thông tin công ty trách nhiệm hữu hạn ...,LUẬT DOANH NGHIỆP
6,nghĩa vụ doanh nghiệp doanh nghiệp xã hội đáp ...,LUẬT DOANH NGHIỆP
77,chủ tịch công ty chủ tịch công ty chủ sở hữu c...,LUẬT DOANH NGHIỆP
...,...,...
1094,tổ chức đối thoại làm việc đối thoại làm việc ...,Luật lao động
1058,kết thúc thời gian thử kết thúc thời gian thử ...,Luật lao động
1210,thẩm quyền giải quyết tranh chấp lao động cá n...,Luật lao động
1140,kỷ luật lao động kỷ luật lao động quy định tuâ...,Luật lao động


In [16]:
le = LabelEncoder()
df_train['label'] = le.fit_transform(df_train['label'])
df_train


,cleaned_text,label
2,áp dụng luật doanh nghiệp luật trường hợp luật...,1
3,bảo đảm nhà nước đối với doanh nghiệp chủ sở h...,1
5,quyền doanh nghiệp tự do kinh doanh ngành nghề...,1
8,đại diện pháp luật doanh nghiệp đại diện pháp ...,1
11,trách nhiệm đại diện ủy quyền chủ sở hữu thành...,1
...,...,...
1227,trường hợp đình công bất hợp pháp trường hợp đ...,7
1228,thông báo quyết định đóng cửa tạm thời làm việ...,7
1229,trường hợp cấm đóng cửa tạm thời làm việc thời...,7
1230,tiền lương quyền lợi hợp pháp lao động thời gi...,7


In [17]:
le = LabelEncoder()
df_test['label'] = le.fit_transform(df_test['label'])
df_test

,cleaned_text,label
62,tiền lương thù lao thưởng lợi ích chủ tịch hội...,1
40,chi nhánh văn phòng đại diện địa điểm kinh doa...,1
93,trách nhiệm chủ tịch thành viên hội đồng thành...,1
18,hồ sơ đăng ký công ty cổ phần giấy đề nghị đăn...,1
81,cơ cấu tổ chức quản lý công ty trách nhiệm hữu...,1
...,...,...
1180,gia hạn thu hồi giấy phép lao động giấy xác nh...,7
1125,khấu trừ tiền lương sử dụng lao động khấu trừ ...,7
1097,nguyên tắc thương lượng tập thể thương lượng t...,7
1092,học nghề tập nghề làm việc sử dụng lao động họ...,7


In [18]:
le = LabelEncoder()
df_val['label'] = le.fit_transform(df_val['label'])
df_val

,cleaned_text,label
32,định giá tài sản góp vốn tài sản góp vốn đồng ...,1
1,đối tượng áp dụng doanh nghiệp cơ quan tổ chức...,1
69,công bố thông tin công ty trách nhiệm hữu hạn ...,1
6,nghĩa vụ doanh nghiệp doanh nghiệp xã hội đáp ...,1
77,chủ tịch công ty chủ tịch công ty chủ sở hữu c...,1
...,...,...
1094,tổ chức đối thoại làm việc đối thoại làm việc ...,7
1058,kết thúc thời gian thử kết thúc thời gian thử ...,7
1210,thẩm quyền giải quyết tranh chấp lao động cá n...,7
1140,kỷ luật lao động kỷ luật lao động quy định tuâ...,7


In [19]:
df_train['label'].value_counts()

label
5    159
7    151
0    122
3     93
8     93
2     65
1     48
6     47
4     13
Name: count, dtype: int64

In [20]:
df_test['label'].value_counts()

label
1    27
4    27
3    27
0    27
6    27
2    27
5    27
8    27
7    27
Name: count, dtype: int64

In [21]:
df_val['label'].value_counts()

label
1    22
4    22
3    22
0    22
6    22
2    22
5    22
8    22
7    22
Name: count, dtype: int64

In [22]:
X_train = df_train['cleaned_text']
y_train = df_train['label']

X_test = df_test['cleaned_text']
y_test = df_test['label']

X_val = df_val['cleaned_text']
y_val = df_val['label']

print(f'Number samples of training set: {X_train.shape}')
print(f'Number samples of test set: {X_val.shape}')
print(f'Number samples of test set: {X_test.shape}')

Number samples of training set: (791,)
Number samples of test set: (198,)
Number samples of test set: (243,)


In [23]:
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(X_train)
X_val = tfidf.transform(X_val)
X_test = tfidf.transform(X_test)

# TRANING MODEL

In [26]:
# Nối 2 dataframe dữ liệu df_train, df_val để dùng cho cross vali
train_df =  pd.concat([df_train, df_val], axis=0)
train_df

,cleaned_text,label
2,áp dụng luật doanh nghiệp luật trường hợp luật...,1
3,bảo đảm nhà nước đối với doanh nghiệp chủ sở h...,1
5,quyền doanh nghiệp tự do kinh doanh ngành nghề...,1
8,đại diện pháp luật doanh nghiệp đại diện pháp ...,1
11,trách nhiệm đại diện ủy quyền chủ sở hữu thành...,1
...,...,...
1094,tổ chức đối thoại làm việc đối thoại làm việc ...,7
1058,kết thúc thời gian thử kết thúc thời gian thử ...,7
1210,thẩm quyền giải quyết tranh chấp lao động cá n...,7
1140,kỷ luật lao động kỷ luật lao động quy định tuâ...,7


## 1. Support Vector Machine (SVM)

### 1.1. Defalt hyperparameters

In [27]:
svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [28]:
X_train_pred = svc.predict(X_train)
X_test_pred = svc.predict(X_test)
print(classification_report(X_train_pred, y_train))
print(classification_report(X_test_pred, y_test))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      0.97      0.98        96
           4       0.92      1.00      0.96        12
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       0.97      0.99      0.98        91

    accuracy                           0.99       791
   macro avg       0.99      1.00      0.99       791
weighted avg       1.00      0.99      0.99       791

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        30
           1       0.96      0.93      0.95        28
           2       0.96      0.96      0.96        27
           3       1.00      1.00      1.00        27
           4       0.56 

In [29]:
print(f'Accuracy: {round(accuracy_score(X_test_pred, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_pred, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_pred, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_pred, y_test, average="weighted"), 4)}')

Accuracy: 0.93
Precision: 0.9523
Recall: 0.93
F1-score: 0.9349


### 1.2 Best hyperparameters

In [30]:
# Định nghĩa các siêu tham số cần tinh chỉnh
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

# Định nghĩa GridSearchCV
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Huấn luyện mô hình
grid_search.fit(X_val, y_val)

# In ra các siêu tham số tốt nhất
print("Best parameters found: ", grid_search.best_params_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters found:  {'C': 1, 'gamma': 1, 'kernel': 'linear'}


In [31]:
svc_hp = SVC( C= 1, gamma = 1, kernel= 'linear')
svc_hp.fit(X_train, y_train)

SVC(C=1, gamma=1, kernel='linear')

In [32]:
X_train_pred_hp = svc_hp.predict(X_train)
X_test_pred_hp = svc_hp.predict(X_test)
print(classification_report(X_train_pred_hp, y_train))
print(classification_report(X_test_pred_hp, y_test))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      0.97      0.98        96
           4       0.92      1.00      0.96        12
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       0.97      0.99      0.98        91

    accuracy                           0.99       791
   macro avg       0.99      1.00      0.99       791
weighted avg       1.00      0.99      0.99       791

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        30
           1       0.96      0.93      0.95        28
           2       0.96      0.96      0.96        27
           3       1.00      1.00      1.00        27
           4       0.78 

In [33]:
print(f'Accuracy: {round(accuracy_score(X_test_pred_hp, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_pred_hp, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_pred_hp, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_pred_hp, y_test, average="weighted"), 4)}')

Accuracy: 0.9547
Precision: 0.9607
Recall: 0.9547
F1-score: 0.9556


## 2. Random Forest

### 2.1 Default hyperparameters

In [35]:
rfc_default = RandomForestClassifier()
rfc_default.fit(X_train, y_train)

RandomForestClassifier()

In [36]:

X_train_pred_rf = rfc_default .predict(X_train)
X_test_pred_rf = rfc_default.predict(X_test)
print(classification_report(X_train_pred_rf, y_train))
print(classification_report(X_test_pred_rf, y_test))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        65
           3       1.00      1.00      1.00        93
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00       159
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       151
           8       1.00      1.00      1.00        93

    accuracy                           1.00       791
   macro avg       1.00      1.00      1.00       791
weighted avg       1.00      1.00      1.00       791

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        28
           1       0.96      0.90      0.93        29
           2       1.00      0.93      0.96        29
           3       1.00      0.90      0.95        30
           4       0.44 

In [37]:
print(f'Accuracy: {round(accuracy_score(X_test_pred_rf, y_test), 4)}')
print(f'Precision: {round(precision_score(X_test_pred_rf, y_test, average="weighted"), 4)}')
print(f'Recall: {round(recall_score(X_test_pred_rf, y_test, average="weighted"), 4)}')
print(f'F1-score: {round(f1_score(X_test_pred_rf, y_test, average="weighted"), 4)}')

Accuracy: 0.9136
Precision: 0.9496
Recall: 0.9136
F1-score: 0.9215


### 2.2 Best hyperparameters

In [40]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': [ 50, 100, 200],
    'max_depth': [None, 50, 100, 150, 200],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 4]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=rfc_default, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X_val, y_val)

# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_

print('f"Best Hyperparameters: {best_params}"')

KeyboardInterrupt: 

In [ ]:
# # Best Hyperparameters: {'max_depth': 100, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
# rfc_best_hp = RandomForestClassifier(max_depth=100, min_samples_leaf=1, min_samples_split=5, n_estimators=200, criterion='entropy')
# rfc_best_hp.fit(X_train, y_train)

In [ ]:
# print(model=rfc_best_hp, feature=X_train, label=y_train, title='Accuracy score of model after hyperparameter tuning in training set', metric1=accuracy_score, metric2=classification_report)
# print('~'*55)
# print(model=rfc_best_hp, feature=X_val, label=y_val, title='Accuracy score of model after hyperparameter tuning in validation set', metric1=accuracy_score, metric2=classification_report)
# print('~'*55)
# print(model=rfc_best_hp, feature=X_test, label=y_test, title='Accuracy score of model after hyperparameter tuning in test set', metric1=accuracy_score, metric2=classification_report)

## 3.Logistic Regression

### 3.1 Defalt hyperparameters

In [ ]:
# log_reg_default = LogisticRegression()
# log_reg_default.fit(X_train, y_train)